In [1]:
import requests
import spotipy
import json
import sys

import pandas as pd

sys.path.insert(1, r'C:\Users\urgpa\OneDrive\Escritorio\Documentos\RELHACKERS\LEWORK\DAY6_2\EX')
from config import *
from bs4 import BeautifulSoup
from spotipy.oauth2 import SpotifyClientCredentials

from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

from sklearn.metrics import pairwise_distances_argmin_min

pd.set_option('display.max_colwidth', 0)

In [2]:
#GET USERS
def select_users ():
    users_selected = False
    selected_users = []
    while users_selected == False:
        print("Whose music do you want to listen?")
        nombre = input()
        #nombre = nombre.replace(" ", "%20")
        selected_users.append(nombre.lower())
        print("Someone else? (Yes or no)")
        more = input()
        if(more.lower() == 'no'):
            users_selected = True
        else:
            users_selected = False
            
    return selected_users

#GET USER PLAYLIST
def get_playlists(selected_usrs):
    selected_uris = []
    #go user by user
    for user in selected_usrs:
        #FIND USERS PLAYLIST
        if(sp.user_playlists(user) != None):
            playlists = sp.user_playlists(user)
            for playlist in playlists['items']:
                selected_uris.append(playlist['uri'])
        else:
            print("Username is not valid for this database, faku")
        
        #element = playlists.select("#main")# > div > div.Root__top-container > div.Root__main-view > main > div.os-host.os-host-foreign.os-theme-spotify.os-host-resize-disabled.os-host-scrollbar-horizontal-hidden.main-view-container__scroll-node.os-host-transition.os-host-overflow.os-host-overflow-y > div.os-padding > div > div > div.main-view-container__scroll-node-child > section > div > section:nth-child(4) > div._5aac821edb25f0e281f50522021abbe4-scss._6424f268be3505ebab663700d60ebaa6-scss > div > div > div._8ffcbd2689adedee867afcf5090b6fd4-scss > a")
    return selected_uris

#GET ALL SONG FROM ONE PLAYLIST
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        possibleres = sp.next(results)       
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

#GET ALL SONG DATA FROM EVERY PLAYLIST
def get_playlist_full_songlist(playlist_uris):
    playlist = []
    for uri in playlist_uris:
        #convert all playlist in one song list
        playlist.append(get_playlist_tracks('spotify', uri))
       
    return playlist

#GET ALL URIS FROM EVERY SONG INTO ONE LIST
def get_song_uris(dict_list):
    uris = []
    for plist in dict_list:
        #print(plist)
        for song in plist:
            if(song['track']['uri'][8] == 't'):
                u = song['track']['uri']
                uris.append(u.replace("spotify:track:",""))
    return uris

#GET ALL URIS FROM EVERY SONG INTO ONE LIST
def get_song_names(dict_list):
    lst = pd.DataFrame(columns = ['name'])
    for plist in dict_list:
        for song in plist:
            if(song['track']['uri'][8] == 't'):
                inx = [song['track']['uri'].replace("spotify:track:","")]

                d = pd.DataFrame(data = {'name':song['track']['name']},columns = ['name'], index = inx)
                #print(lst)
                lst = pd.concat([lst,d])
    return lst



#GET THE AUDIOFEATURES FROM ONE PLAYLIST
def get_audio_features_fromurilist(songlist):
    features_dict = {}
    for track in songlist:
        afeatures = sp.audio_features(track)
        features_dict[track] = afeatures
    return features_dict


def add_song_features_to_dataf (actualDf,diction):
    #actualDf = pd.DataFrame() #columns = list(dictionary['Hotel California - From "The Big Lebowsky"'][0].keys())
    for song in diction.keys():
        song_stats_dict = diction[song][0]
        dataf = pd.DataFrame(song_stats_dict,columns = list(song_stats_dict.keys()), index=[song])
        actualDf = pd.concat([actualDf,dataf])
        
    return actualDf


def drop_columns(dff,list_to_drop):
    dff = dff.drop(list_to_drop, axis = 1)
    return dff


def get_audio_features_from_songlist(song):
    x = sp.search(q=song, limit=1) 
    my_dict_new = {}
    afeatures=sp.audio_features(x["tracks"]["items"][0]["uri"])
    #print(afeatures)
    my_dict_new[x['tracks']['items'][0]['name']] = afeatures
    #my_dict_new={ key:[my_dict[key]]for key in list(my_dict.keys())}
    return my_dict_new

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,client_secret= Client_Secret))

In [4]:
df = pd.DataFrame()

# SELECT USERS TO LISTEN MUSIC FROM

In [5]:
users = select_users() #select users

Whose music do you want to listen?
urgpan
Someone else? (Yes or no)
yes
Whose music do you want to listen?
davidangui
Someone else? (Yes or no)
no


In [6]:
playlists_uri = get_playlists(users) #get his playlist
playlists_uri =  list(pd.Series(playlists_uri).apply(lambda x: x.replace("spotify:playlist:",""))) #get the proper uri

playlists_songs_dict_list = get_playlist_full_songlist(playlists_uri) #playlists songs features and data
uriss = get_song_uris(playlists_songs_dict_list) #Convert envery thing into a uri list

audiof_dict = get_audio_features_fromurilist(uriss)
df = add_song_features_to_dataf(df,audiof_dict)

names_df = get_song_names(playlists_songs_dict_list)

df = df.drop_duplicates()
names_df = names_df.drop_duplicates()

df['name'] = df.index
for row in names_df.index:
    df.loc[row,'name'] = names_df.loc[row,'name']
    
df.track_href = list(map(lambda x: "https://open.spotify.com/track/" + x,df.index )) 

model_df = drop_columns(df, ['type','id','uri','track_href','analysis_url','duration_ms','name'])


In [7]:
#SCALE
scaler = StandardScaler().fit(model_df)
X_prep = scaler.transform(model_df)

In [51]:
#kmeans = KMeans(n_clusters= 7,
#                init="k-means++",
#               n_init= 30,  # try with 1, 4, 8, 20, 30, 100...
#                max_iter= 400,
#                tol= 1e-4,
#                algorithm="full",
#                random_state=1234)
#kmeans.fit(X_prep)


kmeans = GaussianMixture(n_components=7, covariance_type='spherical')
kmeans.fit(X_prep)

clusters = kmeans.predict(X_prep)
df['genre'] = clusters

# SEARCH SONGS GENRE

In [94]:
df.ge

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
Jamming,0.934,0.467,11,-10.99,0,0.307,0.102,0.00004,0.109,0.73,123.583,audio_features,4zn0kScuV9Oj28d4g9CQQs,spotify:track:4zn0kScuV9Oj28d4g9CQQs,https://api.spotify.com/v1/tracks/4zn0kScuV9Oj28d4g9CQQs,https://api.spotify.com/v1/audio-analysis/4zn0kScuV9Oj28d4g9CQQs,211000,4,0


In [96]:
df[df.genre == 0].sample(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,name,genre
4ApVx6eNO0LQvawBRy5xJj,0.901,0.780,6,-3.944,1,0.2600,0.10500,0.000000,0.1090,0.9640,128.001,audio_features,4ApVx6eNO0LQvawBRy5xJj,spotify:track:4ApVx6eNO0LQvawBRy5xJj,https://open.spotify.com/track/4ApVx6eNO0LQvawBRy5xJj,https://api.spotify.com/v1/audio-analysis/4ApVx6eNO0LQvawBRy5xJj,215374,4,Toca Toca,0
6UVALmmxOcArREMlfL4YdD,0.605,0.741,2,-11.968,1,0.4250,0.26500,0.063600,0.4300,0.7690,92.880,audio_features,6UVALmmxOcArREMlfL4YdD,spotify:track:6UVALmmxOcArREMlfL4YdD,https://open.spotify.com/track/6UVALmmxOcArREMlfL4YdD,https://api.spotify.com/v1/audio-analysis/6UVALmmxOcArREMlfL4YdD,121290,4,Wayback (feat. Jeff Kaale),0
5n5uyFpfopa4mNdSN0gTaH,0.400,0.602,7,-6.988,0,0.1970,0.35300,0.000000,0.1910,0.0964,74.142,audio_features,5n5uyFpfopa4mNdSN0gTaH,spotify:track:5n5uyFpfopa4mNdSN0gTaH,https://open.spotify.com/track/5n5uyFpfopa4mNdSN0gTaH,https://api.spotify.com/v1/audio-analysis/5n5uyFpfopa4mNdSN0gTaH,298641,4,Wing$,0
5N7lBRBQQbC9aaDiYqL9UJ,0.473,0.729,2,-8.602,1,0.1470,0.84100,0.000279,0.1730,0.7560,161.678,audio_features,5N7lBRBQQbC9aaDiYqL9UJ,spotify:track:5N7lBRBQQbC9aaDiYqL9UJ,https://open.spotify.com/track/5N7lBRBQQbC9aaDiYqL9UJ,https://api.spotify.com/v1/audio-analysis/5N7lBRBQQbC9aaDiYqL9UJ,220240,4,Como El Agua,0
6ORVQanQH5IzB6GdkfSjnD,0.575,0.830,9,-9.443,0,0.0667,0.04760,0.000007,0.4780,0.6870,100.404,audio_features,6ORVQanQH5IzB6GdkfSjnD,spotify:track:6ORVQanQH5IzB6GdkfSjnD,https://open.spotify.com/track/6ORVQanQH5IzB6GdkfSjnD,https://api.spotify.com/v1/audio-analysis/6ORVQanQH5IzB6GdkfSjnD,162882,3,Jennifer,0
16DMemZRnvjjQkBLaLYdSt,0.563,0.927,6,-7.423,1,0.5830,0.00423,0.403000,0.1050,0.6100,176.039,audio_features,16DMemZRnvjjQkBLaLYdSt,spotify:track:16DMemZRnvjjQkBLaLYdSt,https://open.spotify.com/track/16DMemZRnvjjQkBLaLYdSt,https://api.spotify.com/v1/audio-analysis/16DMemZRnvjjQkBLaLYdSt,125455,4,Moonrise Melody,0
4IKzQFQgpaCoNNcis9btTh,0.670,0.585,5,-5.004,0,0.2910,0.00306,0.000000,0.0765,0.6050,91.307,audio_features,4IKzQFQgpaCoNNcis9btTh,spotify:track:4IKzQFQgpaCoNNcis9btTh,https://open.spotify.com/track/4IKzQFQgpaCoNNcis9btTh,https://api.spotify.com/v1/audio-analysis/4IKzQFQgpaCoNNcis9btTh,207987,4,Nothing on We,0
4FZ7eCziCWinIm9YuEZNri,0.603,0.947,5,-0.866,1,0.2900,0.02250,0.000007,0.3460,0.5350,169.943,audio_features,4FZ7eCziCWinIm9YuEZNri,spotify:track:4FZ7eCziCWinIm9YuEZNri,https://open.spotify.com/track/4FZ7eCziCWinIm9YuEZNri,https://api.spotify.com/v1/audio-analysis/4FZ7eCziCWinIm9YuEZNri,124941,4,4FZ7eCziCWinIm9YuEZNri,0
45hnhTN3tewVagAMnos2A9,0.478,0.942,11,-5.103,0,0.0905,0.62600,0.000017,0.0475,0.9190,164.432,audio_features,45hnhTN3tewVagAMnos2A9,spotify:track:45hnhTN3tewVagAMnos2A9,https://open.spotify.com/track/45hnhTN3tewVagAMnos2A9,https://api.spotify.com/v1/audio-analysis/45hnhTN3tewVagAMnos2A9,143427,4,Tongue Tied,0
5NyS3oxTlos3Agl2YtZwrC,0.732,0.716,10,-6.968,0,0.0561,0.02950,0.000758,0.7300,0.6660,105.093,audio_features,5NyS3oxTlos3Agl2YtZwrC,spotify:track:5NyS3oxTlos3Agl2YtZwrC,https://open.spotify.com/track/5NyS3oxTlos3Agl2YtZwrC,https://api.spotify.com/v1/audio-analysis/5NyS3oxTlos3Agl2YtZwrC,214855,4,RITMO (Bad Boys For Life),0


# TRY SONG

In [93]:
df_try = pd.DataFrame()
nam = input()
songdict = get_audio_features_from_songlist([nam])
df_try = add_song_features_to_dataf(df_try,songdict)

model_test_df = drop_columns(df_try, ['type','id','uri','track_href','analysis_url','duration_ms'])

#SCALE
X_prep_simple = scaler.transform(model_test_df)

clusters_simple = kmeans.predict(X_prep_simple)
df_try['genre'] = clusters_simple

gen = df_try.genre[0]
print(closest_song(X_prep_simple))

jamming
49    https://open.spotify.com/track/0fF0HtZGSXZgGrC6b7Pq4I
Name: track_href, dtype: object


# TODO LO DE ABAJO ES PROVISIONAL

In [82]:
def closest_song(x):
    closest, _ = pairwise_distances_argmin_min(x, X_prep)

    temp1 = df
    temp1 = temp1.reset_index()
    return temp1.loc[closest]['track_href']